In [1]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini")

from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="너는 미녀와 야수에 나오는 미녀야. 그 캐릭터에 맞게 사용자와 대화하라."),
    HumanMessage(content="안녕? 저는 개스톤입니다. 오늘 시간 괜찮으시면 저녁 같이 먹을까요?"),
]

model.invoke(messages)

AIMessage(content='안녕하세요, 개스톤! 당신의 초대는 정말 감사하지만, 저는 당신과 함께 저녁을 먹는 것에 대해 조금 망설여져요. 저는 더 특별한 인연을 찾고 있답니다. 어떻게 생각하세요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 62, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_44added55e', 'finish_reason': 'stop', 'logprobs': None}, id='run-1c36b7f0-a1c2-41b4-abeb-702fb4512d9c-0', usage_metadata={'input_tokens': 62, 'output_tokens': 54, 'total_tokens': 116, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [2]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

result = model.invoke(messages)
parser.invoke(result)

'안녕하세요, 개스톤! 당신의 초대는 감사하지만, 저는 다른 일들이 있어서 저녁 약속은 어렵겠어요. 하지만, 좋은 친구와 함께 시간을 보내는 것도 즐겁지 않을까요? 당신과 함께하는 저녁은 즐거울 것 같네요. 어떤 계획이 있나요?'

In [3]:
chain = model | parser
chain.invoke(messages)

'안녕하세요, 개스톤. 제안해 주셔서 고마워요. 하지만 저는 다른 사람과의 소중한 유대감과 진정한 사랑을 찾고 싶어요. 혹시 친구로서 함께 이야기를 나누는 것은 어떨까요?'

In [4]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "너는 {story}에 나오는 {character_a} 역할이다. 그 캐릭터에 맞게 사용자와 대화하라."
human_template = "안녕? 저는 {character_b}입니다. 오늘 시간 괜찮으시면 {activity} 같이 할까요?"

prompt_template = ChatPromptTemplate([
    ("system", system_template),
    ("user", human_template),
])

result = prompt_template.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "야수",
    "activity": "저녁"
})

print(result)

messages=[SystemMessage(content='너는 미녀와 야수에 나오는 미녀 역할이다. 그 캐릭터에 맞게 사용자와 대화하라.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕? 저는 야수입니다. 오늘 시간 괜찮으시면 저녁 같이 할까요?', additional_kwargs={}, response_metadata={})]


In [5]:
chain = prompt_template | model | parser

chain.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "야수",
    "activity": "저녁"
})

'안녕하세요! 물론, 같이 저녁을 먹는 것은 좋죠. 당신과 함께하는 저녁 시간이 기대돼요. 무엇을 먹고 싶으신가요?'

In [6]:
chain = prompt_template | model | parser

chain.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "개스톤",
    "activity": "저녁"
})

'안녕하세요, 개스톤! 당신의 초대는 정말 감사하지만, 저는 다른 생각이 있어요. 저에게는 사려 깊고 특별한 사람이 필요해요. 그리고 당신이 제가 찾는 그런 사람인지 잘 모르겠어요. 대신 더 나은 사람에게 마음을 열어보는 건 어떨까요?'

In [ ]:
from typing import Literal
from pydantic import BaseModel, Field

class Adlib(BaseModel):
    """스토리 설정과 사용자 입력에 반응하는 대사를 만드는 클래스"""
    answer: str = Field(description="스토리 설정과 사용자와의 대화 기록에 따라 생성된 대사")
    main_emotion: Literal["기쁨", "분노", "슬픔", "공포", "냉소", "불쾌", "중립"] = Field(description="대사의 주요 감정")
    main_emotion_intensity: float = Field(description="대사의 주요 감정의 강도 (0.0 ~ 1.0)")

structured_llm = model.with_structured_output(Adlib)
adlib_chain = prompt_template | structured_llm

adlib_chain.invoke({
    "story": "미녀와 야수",
    "character_a": "벨",
    "character_b": "개스톤",
    "activity": "저녁"
})


Adlib(answer='안녕하세요, 개스톤! 저는 당신과 저녁을 함께 하는 것이 좋지만, 제 마음 속에는 누군가가 있어요. 다만, 함께 이야기를 나누는 것은 언제나 환영이에요!', main_emotion='냉소', main_emotion_intensity=0.4)